In [1]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [2]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

c:\Github\Local\ai-agents-for-beginners_cdc\.venv\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


In [3]:
frontdesk_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.""",
)

concierge_agent = AssistantAgent(
    "concierge_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, respond with 'APPROVE'
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """,
)

In [4]:
termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat(
    [frontdesk_agent, concierge_agent], termination_condition=termination
)

async for message in team.run_stream(task="I would like to plan a trip to Paris."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='4a5efc37-6937-42ce-befe-771722f14d3f' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 2, 14, 3, 54, 6, 454399, tzinfo=datetime.timezone.utc) content='I would like to plan a trip to Paris.' type='TextMessage'
id='b7dc5859-e5d8-4161-a1cc-b3573c283573' source='planner_agent' models_usage=RequestUsage(prompt_tokens=100, completion_tokens=6) metadata={} created_at=datetime.datetime(2026, 2, 14, 3, 54, 8, 198001, tzinfo=datetime.timezone.utc) content='Visit the Louvre Museum.' type='TextMessage'
id='385b22be-5e1c-462b-8068-f4ecb3877e57' source='concierge_agent' models_usage=RequestUsage(prompt_tokens=107, completion_tokens=83) metadata={} created_at=datetime.datetime(2026, 2, 14, 3, 54, 10, 206509, tzinfo=datetime.timezone.utc) content="While the Louvre is a world-renowned museum and a must-see for many visitors, it can also be quite touristy. To enhance the experience for a traveler looking for a more local and authentic feel, consider recommending a visit


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
